In [187]:
import pandas as pd
import numpy as np
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", 200)
pd.set_option("display.precision", 2)
from datetime import datetime
from datetime import date
import plotly.graph_objects as go
import plotly.express as px
import holidays

In [161]:
data_file_path = "data/even_route.csv"
clean_file = "/tmp/clean.tmp"

In [162]:
# We need to do this because the directions can have , in them so it would
# look like the data is an invalid csv if we didn't do this
with open(data_file_path, 'r') as data_file:
    with open(clean_file, 'w') as rewrite:
        for line in data_file:
            columns = line.split(",")
            columns = columns[0:3] + columns[4:5] + [columns[0] + " " + columns[1]]
            new_line = ",".join(columns)
            rewrite.write(new_line + "\n")

In [179]:
df = pd.read_csv(clean_file, names=[
                 "date", "time", "weekday", "length", "datetime"])
df['datetime'] = pd.to_datetime(df['datetime'])
# df[(df.length > 90)]

In [200]:
# Filter out the errors
df = df[(df.length > 20) & (df.length < 90)]
df = df[(df.time.str[-1] == "0") | (df.time.str[-1] == "5")]

# Filter out holidays and randomly bad days
# Seems like there was a bad accident on this day ¯\_(ツ)_/¯
df = df[(df["date"] != "2019-08-27")]
df = df[(df["date"] != "2018-03-08")]
df = df[(df["date"] != "2018-05-23")]
df = df[(df["date"] != "2018-08-07")]
df = df[(df["date"] != "2018-02-13")]
df = df[(df["date"] != "2018-10-23")]
df = df[(df["date"] != "2019-04-09")]

us_holidays = holidays.US()
def is_good_day(dt):
    if dt in us_holidays:
        return False
    day = dt.date()
    # Holiday season
    if day.month == 12 and day.day > 10:
        return False
    # Near Thanksgiving
    if day.month == 11 and day.day > 22:
        return False
    if day.month == 1 and day.day < 15:
        return False
    return True

df = df[df["datetime"].apply(is_good_day)]

In [194]:
start = datetime(2010, 1, 1)
end = datetime.now()
grouped = df[(df["datetime"] > start) & (df["datetime"] <= end)].groupby(['weekday', 'time']).mean()
grouped.reset_index(inplace=True)
grouped["weekday"] = pd.Categorical(grouped["weekday"])
fig = px.scatter(grouped, x="time", y="length", color="weekday")
fig.show()

In [201]:
start = datetime(2010, 1, 1)
end = datetime.now()
all_between_dates = df[((df["datetime"] > start) & (df["datetime"] <= end)) & ((df["time"] == "17:00") | (df["time"] == "08:00"))]
fig = px.scatter(all_between_dates, x="datetime", y="length")
fig.show()

In [203]:
# View data on a single day
df[(df["date"] == "2019-04-09")]

,date,time,weekday,length,datetime


In [204]:
# Try and come up with a graph for showing difference year vs. year

In [205]:
# Try and come up with a graph for showing the difference between two months (or seasons?)